In [36]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.frequent_patterns import fpmax, fpgrowth
import pandas as pd

In [51]:
df = pd.read_csv('Groceries data train.csv')
df.head()

,User_id,Date,itemDescription,year,month,day,day_of_week
0,2351.0,1/01/2014,cleaner,2014.0,1.0,1.0,2.0
1,2226.0,1/01/2014,sausage,2014.0,1.0,1.0,2.0
2,1922.0,1/01/2014,tropical fruit,2014.0,1.0,1.0,2.0
3,2943.0,1/01/2014,whole milk,2014.0,1.0,1.0,2.0
4,1249.0,1/01/2014,citrus fruit,2014.0,1.0,1.0,2.0


In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26985 entries, 0 to 26984
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   User_id          19382 non-null  float64
 1   Date             19382 non-null  object 
 2   itemDescription  19382 non-null  object 
 3   year             19382 non-null  float64
 4   month            19382 non-null  float64
 5   day              19382 non-null  float64
 6   day_of_week      19382 non-null  float64
dtypes: float64(5), object(2)
memory usage: 1.4+ MB


In [53]:

# data.fillna(0, inplace=True)
df.fillna('', inplace=True)


<ipython-input-53-477602e153ab>:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('', inplace=True)


In [ ]:
# Grouping the Data into Transactions by User_id and Date
grouped = df.groupby(['User_id', 'Date'])['itemDescription'].apply(list).reset_index()

# Creating list of transactions
transactions = grouped['itemDescription'].tolist()

print("Total number of transactions:", len(transactions))
print("\nSample transactions")
for t in transactions[:10]:
    print(t)

Total number of transactions: 8362

Sample transactions
['whole milk', 'pastry', 'salty snack']
['whole milk', 'soda']
['sausage', 'whole milk', 'rolls/buns']
['frankfurter', 'soda']
['frozen vegetables', 'other vegetables']
['butter', 'whole milk']
['root vegetables', 'detergent']
['frozen meals', 'dental care']
['rolls/buns', 'rolls/buns']
['whole milk', 'chocolate', 'packaged fruit/vegetables', 'rolls/buns']


In [55]:
grouped.head()

,User_id,Date,itemDescription
0,1000.0,24/06/2014,"[whole milk, pastry, salty snack]"
1,1001.0,12/12/2014,"[whole milk, soda]"
2,1001.0,2/07/2014,"[sausage, whole milk, rolls/buns]"
3,1001.0,20/01/2015,"[frankfurter, soda]"
4,1002.0,2/09/2014,"[frozen vegetables, other vegetables]"


In [56]:
grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8362 entries, 0 to 8361
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   User_id          8362 non-null   object
 1   Date             8362 non-null   object
 2   itemDescription  8362 non-null   object
dtypes: object(3)
memory usage: 196.1+ KB


In [57]:
#One-Hot Encode the Transaction Data
te = TransactionEncoder()
te_array = te.fit(transactions).transform(transactions)
df_encoded = pd.DataFrame(te_array, columns=te.columns_)


In [ ]:
# Frequent Itemset Mining 

frequent_itemsets_apriori = apriori(df_encoded, min_support=0.03, use_colnames=True)

if frequent_itemsets_apriori.empty:
    print("No frequent itemsets found.")
else:
    print("Frequent Itemsets:")
    print(frequent_itemsets_apriori)

    #Generate Association Rules
    rules_apriori = association_rules(frequent_itemsets_apriori, metric="confidence", min_threshold=0.02)
    print("Association Rules:")
    print(rules_apriori)

Frequent Itemsets:
     support                 itemsets
0   0.040541           (bottled beer)
1   0.060751          (bottled water)
2   0.040182            (brown bread)
3   0.034083                 (butter)
4   0.034202            (canned beer)
5   0.034561           (citrus fruit)
6   0.030615                 (coffee)
7   0.030136                   (curd)
8   0.039703          (domestic eggs)
9   0.036714  (fruit/vegetable juice)
10  0.035398              (margarine)
11  0.041378             (newspapers)
12  0.106673       (other vegetables)
13  0.049629                 (pastry)
14  0.037551              (pip fruit)
15  0.101890             (rolls/buns)
16  0.060392        (root vegetables)
17  0.052739          (shopping bags)
18  0.094595                   (soda)
19  0.048792         (tropical fruit)
20  0.044367     (whipped/sour cream)
21  0.130471             (whole milk)
22  0.078570                 (yogurt)
Association Rules:
Empty DataFrame
Columns: [antecedents, consequents

In [83]:
# Frequent Itemset Mining using FP-growth
frequent_itemsets_fp = fpgrowth(df_encoded, min_support=0.05, use_colnames=True)
if frequent_itemsets_fp.empty:
    print("No frequent itemsets found.")
else:
    print("Frequent Itemsets (FP-growth):")
    print(frequent_itemsets_fp)

    rules_fp = association_rules(frequent_itemsets_fp, metric="confidence", min_threshold=0.2)
    print("Association Rules:")
    print(rules_fp)


Frequent Itemsets (FP-growth):
    support            itemsets
0  0.130471        (whole milk)
1  0.094595              (soda)
2  0.101890        (rolls/buns)
3  0.106673  (other vegetables)
4  0.060392   (root vegetables)
5  0.052739     (shopping bags)
6  0.060751     (bottled water)
7  0.078570            (yogurt)
Association Rules:
Empty DataFrame
Columns: [antecedents, consequents, antecedent support, consequent support, support, confidence, lift, representativity, leverage, conviction, zhangs_metric, jaccard, certainty, kulczynski]
Index: []
